In [7]:
import pandas as pd
import os 

directory = 'data/'

data_frames = []

for category in os.listdir(directory):
    category_path = os.path.join(directory, category)
    if os.path.isdir(category_path):

        csv_files = [file for file in os.listdir(category_path) if file.endswith('.csv')]

        for file in csv_files:
            file_path = os.path.join(category_path, file)
            df = pd.read_csv(file_path)
            data_frames.append(df)
            
combined_df = pd.concat(data_frames, ignore_index=True)